In [9]:
import os
import sys
import time
import scipy.io as sio
import numpy as np
import scipy
import keras
import PIL.Image as Image
from sklearn.utils import shuffle

In [46]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding3D, BatchNormalization, Flatten, Conv3D, AveragePooling3D, MaxPooling3D, GlobalMaxPooling3D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from Resnet_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
from keras.regularizers import l2
from sklearn.preprocessing import OneHotEncoder
# import keras.backend as K
# K.set_image_data_format('channels_last')
# K.set_learning_phase(1)

In [47]:
uPavia = sio.loadmat('C:\\Users\\de991521\\Desktop\\EE_297_PROJECT\\pavia\\PaviaU.mat')
gt_uPavia = sio.loadmat('C:\\Users\\de991521\\Desktop\\EE_297_PROJECT\\pavia\\PaviaU_gt.mat')
data_IN = uPavia['paviaU']
gt_IN = gt_uPavia['paviaU_gt']
print(data_IN.shape, gt_IN.shape)
data_IN = np.moveaxis(data_IN, 2, 0)
print(data_IN.shape, gt_IN.shape)

(610, 340, 103) (610, 340)
(103, 610, 340) (610, 340)


In [48]:
values,counts = np.unique(gt_IN, return_counts=True)
values,counts

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([164624,   6631,  18649,   2099,   3064,   1345,   5029,   1330,
          3682,    947], dtype=int64))

In [49]:
counts[0] = 0
counts

array([    0,  6631, 18649,  2099,  3064,  1345,  5029,  1330,  3682,
         947], dtype=int64)

In [39]:
np.argmax(counts[1:])

1

In [13]:
len(data_IN[0][0])

340

# Old sample selection method

In [186]:
small_segmented_1 = []
small_seg_gt_1 = []
Cube_size = 50
stride = 1
for i in range(Cube_size-1,len(data_IN[0]), stride):
    for j in range(Cube_size-1,len(data_IN[0][0]), stride):
        if gt_IN[(i-Cube_size+1) + int((Cube_size-1)/2+1)][(j-Cube_size+1) + int((Cube_size-1)/2+1)] != 0:
            small_segmented_1.append(np.asarray(data_IN[:, (i-Cube_size+1):i+1, (j-Cube_size+1):j+1]))
            small_seg_gt_1.append(gt_IN[(i-Cube_size+1)+int(((Cube_size-1)/2)+1)][(j-Cube_size+1)+int(((Cube_size-1)/2)+1)])
small_segmented_1 = np.array(small_segmented_1)
small_seg_gt_1 = np.array(small_seg_gt_1)
small_segmented_1.shape, small_seg_gt_1.shape

((32671, 103, 50, 50), (32671,))

# New sample selection method

In [6]:
def pick_class(Class , Cube_size , Data, Gt, small_segmented_1, small_seg_gt_1):
    indx_class = np.where(gt_IN == Class)
    all_indx = [[indx_class[0][i],indx_class[1][i]] for i in range(len(indx_class[0])) if len(Gt)-np.ceil(Cube_size/2)>indx_class[0][i]>np.ceil(Cube_size/2) and len(Gt[0])-np.ceil(Cube_size/2)>indx_class[1][i]>np.ceil(Cube_size/2)]
    lst = []
    small_segmented_1.append(np.array(Data[:, all_indx[0][0]-int(Cube_size/2):all_indx[0][0] + int(Cube_size/2), (all_indx[0][1]-int(Cube_size/2)):all_indx[0][1]+int(Cube_size/2)]))
    small_seg_gt_1.append(Class)
    lst.append([all_indx[0][0], all_indx[0][1]])
    for i in range(1, len(all_indx)):
        dist = []
        for k in range(len(lst)):
            d = math.sqrt((all_indx[i][0]-lst[k][0])**2 + (all_indx[i][1]-lst[k][1])**2)
            dist.append(d)
        if np.min(dist) > int(Cube_size/2):
            small_segmented_1.append(np.array(Data[:, all_indx[i][0]-int(Cube_size/2):all_indx[i][0] + int(Cube_size/2), (all_indx[i][1]-int(Cube_size/2)):all_indx[i][1]+int(Cube_size/2)]))
            small_seg_gt_1.append(Class)
            lst.append([all_indx[i][0], all_indx[i][1]])
    return small_segmented_1, small_seg_gt_1, lst

def pick_n_class(range_of_class, Cube_size = int, Data = data_IN, Gt=gt_IN, small_segmented_1 = [], small_seg_gt_1 = []):
    class_len = []
    for i in range_of_class:
        
        small_segmented_1, small_seg_gt_1, lst = pick_class(i, Cube_size, data_IN, gt_IN, small_segmented_1, small_seg_gt_1)
        class_len.append(len(lst))
    small_segmented_1 = np.array(small_segmented_1)
    small_seg_gt_1 = np.array(small_seg_gt_1)
    return small_segmented_1, small_seg_gt_1, class_len

# Majority pixel

In [50]:
def pick_class_majority_pixel(Cube_size, Data, Gt, Overlap_Ratio):
    small_segmented_1 = []
    small_seg_gt_1 = []
    stride = int(Cube_size * (1-Overlap_Ratio))
    for i in range(Cube_size-1,len(Data[0]), stride):
        for j in range(Cube_size-1,len(Data[0][0]), stride):
            values,counts = np.unique(Gt[(i-Cube_size+1):i+1, (j-Cube_size+1):j+1], return_counts=True)
            small_segmented_1.append(np.asarray(data_IN[:, (i-Cube_size+1):i+1, (j-Cube_size+1):j+1]))
            if values[np.argmax(counts)] != 0:
                small_seg_gt_1.append(values[np.argmax(counts)])
            else:
                counts[0] = 0
                small_seg_gt_1.append(values[np.argmax(counts)])
    small_segmented_1 = np.array(small_segmented_1)
    small_seg_gt_1 = np.array(small_seg_gt_1)
    
    values_,class_len = np.unique(small_seg_gt_1, return_counts=True)
    return small_segmented_1, small_seg_gt_1, class_len, values_

In [52]:
small_segmented_1, small_seg_gt_1, class_len, values_ = pick_class_majority_pixel(25, data_IN, gt_IN, 0)
class_len, values_

(array([81, 54, 56, 18, 32, 12, 13,  7, 30,  9], dtype=int64),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8))

In [7]:
small_segmented_1, small_seg_gt_1, class_len = pick_n_class(range(1,10), 25, Data = data_IN, Gt=gt_IN, small_segmented_1 = [], small_seg_gt_1 = [])
small_segmented_1.shape, small_seg_gt_1_.shape

NameError: name 'small_seg_gt_1_' is not defined

In [ ]:
class_len

# Test Code

In [227]:
indx_class = np.where(gt_IN == 1)
all_indx = [[indx_class[0][i],indx_class[1][i]] for i in range(len(indx_class[0])) if len(gt_IN)-np.ceil(25/2)>indx_class[0][i]>np.ceil(25/2) and len(gt_IN[0])-np.ceil(25/2)>indx_class[1][i]>np.ceil(25/2)]
lst = [[all_indx[0][0],all_indx[0][1]]]

for i in range(1, len(all_indx)):
    dist = []
    for k in range(len(lst)):
        d = math.sqrt((all_indx[i][0]-lst[k][0])**2 + (all_indx[i][1]-lst[k][1])**2)
        dist.append(d)
    if np.min(dist) >= 12:
        lst.append([all_indx[i][0],all_indx[i][1]])
print(len(lst), len(all_indx))

j = 0
for i in range(len(all_indx)):
    if lst[j] == all_indx[i]:
        print(lst[j], all_indx[i])
        j+=1
    else:
        print(all_indx[i])

118 5806
[14, 99] [14, 99]
[14, 100]
[14, 101]
[14, 102]
[14, 103]
[14, 104]
[14, 109]
[14, 110]
[14, 111] [14, 111]
[14, 112]
[15, 99]
[15, 100]
[15, 101]
[15, 102]
[15, 103]
[15, 104]
[15, 109]
[15, 110]
[15, 111]
[15, 112]
[16, 100]
[16, 101]
[16, 102]
[16, 103]
[16, 104]
[16, 105]
[16, 106]
[16, 107]
[16, 108]
[16, 109]
[16, 110]
[16, 111]
[16, 112]
[17, 101]
[17, 102]
[17, 103]
[17, 104]
[17, 105]
[17, 106]
[17, 107]
[17, 108]
[17, 109]
[17, 110]
[17, 111]
[17, 112]
[18, 101]
[18, 102]
[18, 103]
[18, 104]
[18, 105]
[18, 106]
[18, 107]
[18, 108]
[18, 109]
[18, 110]
[18, 111]
[18, 112]
[18, 113]
[19, 102]
[19, 103]
[19, 104]
[19, 105]
[19, 106]
[19, 107]
[19, 108]
[19, 109]
[19, 110]
[19, 111]
[19, 112]
[19, 113]
[20, 102]
[20, 103]
[20, 104]
[20, 105]
[20, 106]
[20, 107]
[20, 108]
[20, 109]
[20, 110]
[20, 111]
[20, 112]
[20, 113]
[21, 90]
[21, 91]
[21, 92]
[21, 93]
[21, 94]
[21, 95]
[21, 96]
[21, 103]
[21, 104]
[21, 105]
[21, 106]
[21, 107]
[21, 108]
[21, 109]
[21, 110]
[21, 111]
[

[90, 142]
[90, 143]
[90, 144]
[90, 145]
[90, 146]
[90, 147]
[90, 148]
[91, 141]
[91, 142]
[91, 143]
[91, 144]
[91, 145]
[91, 146]
[91, 147]
[91, 148]
[91, 149]
[92, 142]
[92, 143]
[92, 144]
[92, 145]
[92, 146]
[92, 147]
[92, 148]
[92, 149]
[92, 150]
[93, 142]
[93, 143]
[93, 144]
[93, 145]
[93, 146]
[93, 147]
[93, 148]
[93, 149]
[93, 150]
[94, 143]
[94, 144]
[94, 145]
[94, 146]
[94, 147]
[94, 148]
[94, 149]
[94, 150]
[94, 151]
[95, 143]
[95, 144]
[95, 145]
[95, 146]
[95, 147]
[95, 148]
[95, 149]
[95, 150]
[95, 151]
[96, 144]
[96, 145]
[96, 146]
[96, 147]
[96, 148]
[96, 149]
[96, 150]
[96, 151]
[97, 144]
[97, 145]
[97, 146]
[97, 147]
[97, 148]
[97, 149]
[97, 150]
[97, 151]
[97, 152]
[98, 145]
[98, 146]
[98, 147]
[98, 148]
[98, 149]
[98, 150]
[98, 151]
[98, 152]
[98, 153]
[99, 146]
[99, 147]
[99, 148]
[99, 149]
[99, 150]
[99, 151]
[99, 152] [99, 152]
[99, 153]
[99, 154]
[100, 147]
[100, 148]
[100, 149]
[100, 150]
[100, 152]
[100, 153]
[100, 154]
[101, 148]
[101, 149]
[101, 152]
[101, 153]

[235, 210]
[235, 211]
[235, 212]
[235, 213] [235, 213]
[236, 33]
[236, 34]
[236, 208]
[236, 209]
[236, 210]
[236, 211]
[236, 212]
[237, 33]
[237, 34]
[237, 209]
[237, 210]
[237, 211]
[238, 33]
[238, 34]
[238, 35]
[238, 209]
[238, 210]
[238, 211]
[239, 34]
[239, 35]
[239, 36]
[240, 35] [240, 35]
[240, 36]
[241, 35]
[241, 36]
[241, 37]
[242, 35]
[242, 36]
[242, 37]
[242, 215]
[242, 216]
[242, 217]
[243, 36]
[243, 37]
[243, 38]
[243, 212]
[243, 213]
[243, 214]
[243, 215]
[243, 216]
[243, 217]
[244, 37]
[244, 38]
[244, 212]
[244, 213]
[244, 214]
[244, 215]
[244, 216]
[244, 217]
[244, 218]
[245, 37]
[245, 38]
[245, 39]
[245, 212]
[245, 213]
[245, 214]
[245, 215]
[245, 216]
[245, 217]
[245, 218]
[245, 219]
[246, 38]
[246, 39]
[246, 213]
[246, 214]
[246, 215]
[246, 216]
[246, 217]
[246, 218] [246, 218]
[246, 219]
[247, 38]
[247, 39]
[247, 40]
[247, 213]
[247, 214]
[247, 215]
[247, 216]
[247, 217]
[247, 218]
[247, 219]
[248, 39]
[248, 40]
[248, 214]
[248, 215]
[248, 216]
[248, 217]
[248, 218]


[328, 83]
[328, 84]
[328, 257]
[328, 258]
[328, 259]
[328, 260]
[328, 261]
[329, 82]
[329, 83]
[329, 84]
[329, 257]
[329, 258]
[329, 259]
[329, 260]
[329, 261]
[329, 262]
[330, 83]
[330, 84]
[330, 85]
[330, 258] [330, 258]
[330, 259]
[330, 260]
[330, 261]
[330, 262]
[330, 263]
[331, 84]
[331, 85]
[331, 86]
[331, 259]
[331, 260]
[331, 261]
[331, 262]
[331, 263]
[332, 85]
[332, 86]
[332, 87]
[332, 260]
[332, 261]
[332, 262]
[332, 263]
[332, 264]
[333, 85]
[333, 86]
[333, 87]
[333, 260]
[333, 261]
[333, 262]
[333, 263]
[333, 264]
[333, 265]
[334, 86]
[334, 87]
[334, 88]
[334, 260]
[334, 261]
[334, 262]
[334, 263]
[334, 264]
[334, 265]
[335, 86]
[335, 87] [335, 87]
[335, 88]
[335, 261]
[335, 262]
[335, 263]
[335, 264]
[335, 265]
[336, 87]
[336, 88]
[336, 89]
[336, 262]
[336, 263]
[336, 264]
[336, 265]
[336, 266]
[337, 88]
[337, 89]
[337, 262]
[337, 263]
[337, 264]
[337, 265]
[337, 266]
[338, 88]
[338, 89]
[338, 90]
[338, 263]
[338, 264]
[338, 265]
[338, 266]
[338, 267] [338, 267]
[339, 89]

[417, 314]
[417, 315]
[417, 316]
[417, 317]
[417, 318]
[417, 319]
[417, 320]
[417, 321]
[417, 322]
[418, 141]
[418, 142]
[418, 143]
[418, 144]
[418, 278] [418, 278]
[418, 279]
[418, 313]
[418, 314]
[418, 315]
[418, 316]
[418, 317]
[418, 318]
[418, 319]
[418, 320]
[418, 321]
[418, 322]
[419, 142]
[419, 143]
[419, 144]
[419, 278]
[419, 314]
[419, 315]
[419, 316]
[419, 317]
[419, 318]
[419, 319]
[419, 320]
[419, 321]
[419, 322]
[419, 323]
[420, 142]
[420, 143]
[420, 144]
[420, 278]
[420, 314]
[420, 315]
[420, 316]
[420, 317]
[420, 318]
[420, 319]
[420, 320]
[420, 321]
[420, 322]
[420, 323]
[421, 142]
[421, 143]
[421, 144]
[421, 145]
[421, 278]
[421, 315] [421, 315]
[421, 316]
[421, 317]
[421, 318]
[421, 319]
[421, 320]
[421, 321]
[421, 322]
[421, 323]
[421, 324]
[422, 143]
[422, 144]
[422, 145]
[422, 146]
[422, 276]
[422, 277]
[422, 278]
[422, 315]
[422, 316]
[422, 317]
[422, 318]
[422, 319]
[422, 320]
[422, 321]
[422, 322]
[422, 323]
[422, 324]
[423, 144]
[423, 145]
[423, 146]
[423, 147]

[524, 164]
[524, 165]
[525, 164]
[525, 165]
[526, 164]
[526, 165]
[527, 163]
[527, 164]
[528, 163]
[528, 164]
[529, 162]
[529, 163]
[529, 164]
[530, 162]
[530, 163]
[530, 164]
[531, 162]
[531, 163]
[532, 161] [532, 161]
[532, 162]
[532, 163]
[533, 160]
[533, 161]
[533, 162]
[534, 159]
[534, 160]
[534, 161]
[534, 162]
[535, 158]
[535, 159]
[535, 160]
[535, 161]
[535, 162]
[536, 158]
[536, 159]
[536, 160]
[536, 161]
[537, 157]
[537, 158]
[537, 159]
[537, 160]
[538, 156]
[538, 157]
[538, 158]
[538, 159]
[538, 160]
[539, 156]
[539, 157]
[539, 158]
[539, 159]
[540, 155]
[540, 156]
[540, 157]
[540, 158]
[540, 159]
[541, 155]
[541, 156]
[541, 157]
[541, 158]
[542, 154] [542, 154]
[542, 155]
[542, 156]
[542, 157]
[542, 158]
[543, 154]
[543, 155]
[543, 156]
[543, 157]
[544, 155]
[544, 156]
[545, 155]
[545, 156]
[546, 154]
[546, 155]
[547, 153]
[547, 154]
[548, 14] [548, 14]
[548, 152]
[548, 153]
[548, 154]
[549, 14]
[549, 151]
[549, 152]
[549, 153]
[550, 14]
[550, 15]
[550, 150]
[550, 151]
[550

IndexError: list index out of range

In [176]:
values,counts = np.unique(small_seg_gt_1, return_counts=True)
values,counts

(array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([115, 139,  30, 133,  22,  44,  17,  96,  39], dtype=int64))

In [ ]:
s = np.arange(small_segmented_1.shape[0])
print(s)
np.random.shuffle(s)
print(s)
s = np.arange(small_segmented_1.shape[0])
np.random.shuffle(s)
small_segmented_1_s = small_segmented_1[s]
small_seg_gt_1_s = small_seg_gt_1[s]
Lowest_number = np.min(counts)
Lowest_number

In [41]:
a = 80
indx1 = np.where(small_seg_gt_1_s == 1)[0][:Lowest_number-a]
indxt1 =np.where(small_seg_gt_1_s == 1)[0][Lowest_number-a:]
indx2 = np.where(small_seg_gt_1_s == 2)[0][:Lowest_number-a]
indxt2 =  np.where(small_seg_gt_1_s == 2)[0][Lowest_number-a:]
indx3 = np.where(small_seg_gt_1_s == 3)[0][:Lowest_number-a]
indxt3 =  np.where(small_seg_gt_1_s == 3)[0][Lowest_number-a:]
indx4 = np.where(small_seg_gt_1_s == 4)[0][:Lowest_number-a]
indxt4 =  np.where(small_seg_gt_1_s == 4)[0][Lowest_number-a:]
indx5 = np.where(small_seg_gt_1_s == 5)[0][:Lowest_number-a]
indxt5 =  np.where(small_seg_gt_1_s == 5)[0][Lowest_number-a:]
indx6 = np.where(small_seg_gt_1_s == 6)[0][:Lowest_number-a]
indxt6 =  np.where(small_seg_gt_1_s == 6)[0][Lowest_number-a:]
indx7 = np.where(small_seg_gt_1_s == 7)[0][:Lowest_number-a]
indxt7 =  np.where(small_seg_gt_1_s == 7)[0][Lowest_number-a:]
indx8 = np.where(small_seg_gt_1_s == 8)[0][:Lowest_number-a]
indxt8 =  np.where(small_seg_gt_1_s == 8)[0][Lowest_number-a:]
indx9 = np.where(small_seg_gt_1_s == 9)[0][:Lowest_number-a]
indxt9 =  np.where(small_seg_gt_1_s == 9)[0][Lowest_number-a:]
indx1.shape, indx2.shape, indx3.shape, indx4.shape, indx5.shape, indx6.shape, indx7.shape, indx8.shape, indx9.shape

((700,), (700,), (700,), (700,), (700,), (700,), (700,), (700,), (700,))

In [222]:
def train_test_split(percentage, class_len, Data, Gt):
    Xtrain = []
    Xtest = []
    Ytrain = []
    Ytest = []
    class_division = [0]
    c = 0
    for i in range(len(class_len)):
        class_division.append(int(class_len[i]*(percentage/100)) + c)
        class_division.append(class_len[i]+c)
        c = class_len[i]+c
    for i in range(1, len(class_division)):
        if i%2!=0:
            for j in range(class_division[i-1], class_division[i]):
                Xtrain.append(Data[j])
                Ytrain.append(Gt[j])
        else:
            for k in range(class_division[i-1], class_division[i]):
                Xtest.append(Data[k])
                Ytest.append(Gt[k])    
    Xtrain = np.array(Xtrain)
    Xtest = np.array(Xtest)
    Ytrain = np.array(Ytrain)
    Ytest = np.array(Ytest)
    s = np.arange(Xtrain.shape[0])
    np.random.shuffle(s)
    Xtrain = Xtrain[s]
    Ytrain = Ytrain[s]
    s = np.arange(Xtest.shape[0])
    np.random.shuffle(s)
    Xtest = Xtest[s]
    Ytest = Ytest[s]
    Xtrain = np.expand_dims(Xtrain,axis=4) 
    Xtest = np.expand_dims(Xtest,axis=4) 
    Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape
    

    onehot_encoder = OneHotEncoder(sparse=False)
    Ytrain = Ytrain.reshape(len(Ytrain), 1)
    Ytest = Ytest.reshape(len(Ytest),1)
    Ytrain = onehot_encoder.fit_transform(Ytrain)
    Ytest = onehot_encoder.fit_transform(Ytest)
    return Xtrain, Xtest, Ytrain, Ytest

In [223]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(80, class_len, small_segmented_1, small_seg_gt_1)

C:\Users\de991521\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\de991521\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warnin

In [224]:
Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape

((505, 103, 24, 24, 1), (130, 103, 24, 24, 1), (505, 9), (130, 9))

In [42]:
Xtrain = []
Xtest = []
Ytrain = []
Ytest = []
list_of_indx = [indx1,indx2,indx3,indx4,indx5,indx6,indx7,indx8,indx9]
list_of_indxt = [indxt1,indxt2,indxt3,indxt4,indxt5,indxt6,indxt7,indxt8,indxt9]
for i in range(9):
    for j in list_of_indx[i]:
        Xtrain.append(small_segmented_1_s[j])
        Ytrain.append(small_seg_gt_1_s[j])
    for k in list_of_indxt[i]:
        Xtest.append(small_segmented_1_s[k])
        Ytest.append(small_seg_gt_1_s[k])

Xtrain = np.array(Xtrain)
Xtest = np.array(Xtest)
Ytrain = np.array(Ytrain)
Ytest = np.array(Ytest)
Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape

((6300, 103, 50, 50), (26371, 103, 50, 50), (6300,), (26371,))

In [44]:
Xtrain = np.expand_dims(Xtrain,axis=4) 
Xtest = np.expand_dims(Xtest,axis=4) 
Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape

((6300, 103, 50, 50, 1), (26371, 103, 50, 50, 1), (6300,), (26371,))

In [45]:
np.unique(Ytrain, return_counts=True), np.unique(Ytest, return_counts=True)

((array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
  array([700, 700, 700, 700, 700, 700, 700, 700, 700], dtype=int64)),
 (array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
  array([ 4348, 11120,   394,  1871,   645,  4329,   630,  2861,   173],
        dtype=int64)))

In [46]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
Ytrain = Ytrain.reshape(len(Ytrain), 1)
Ytest = Ytest.reshape(len(Ytest),1)
Ytrain = onehot_encoder.fit_transform(Ytrain)
Ytest = onehot_encoder.fit_transform(Ytest)
Ytrain[0], Ytest[0]

C:\Users\devang1997\software\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\devang1997\software\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categori

(array([0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0.]))

In [3]:
def SR_Unit(X, filters):
    # Save the input value
    X_shortcut = X
    print(X.shape)
    X = Conv3D(filters, (1, 1, 1), padding = "same")(X)
    print('--------')
    print(X.shape)
    X = BatchNormalization(axis = 4)(X)
    X = Activation('relu')(X)
    print(X.shape)

    X = Conv3D(filters, kernel_size = (1, 3, 3), padding = 'same')(X)
    X = BatchNormalization(axis = 4)(X)
    X = Activation('relu')(X)
    print(X.shape)


    X = Conv3D(filters, kernel_size = (3, 1, 1), padding = "same")(X)
    X = BatchNormalization(axis = 4)(X)
    X = Activation('relu')(X)
    print(X.shape)

    
    
    X = Conv3D(2*filters, kernel_size = (1, 1, 1), padding = "same")(X)
    X = BatchNormalization(axis = 4)(X)
    print(X.shape)

    
    X_shortcut = Conv3D(filters = 2*filters, kernel_size = (3, 3, 3), padding = 'same')(X_shortcut)
    X_shortcut = BatchNormalization(axis = 4)(X_shortcut)


    X = Add()([X, X_shortcut])
    print(X.shape)
    print('--------')
    X = Activation('relu')(X)
    
    
    return X

In [4]:
def feature_extraction(Sample):
    X = Sample
    X = Conv3D(32, (8, 3, 3), strides=(2,2,2), name='conv1')(X)
    print(X.shape)
    X = BatchNormalization(axis=4, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling3D((1, 2, 2), strides=None)(X)
    print(X.shape)
    # Stage 2
    F1 = 16
    X = SR_Unit(X, filters=F1)
    print(X.shape)
    F2 = 32
    X = SR_Unit(X, filters=F2)
    print(X.shape)
#     F3 = 64
#     X = SR_Unit(X, filters=F3)
#     print(X.shape)
#     F4 = 128
#     X = SR_Unit(X, filters=F4)
    print(X.shape)
    
    return X

In [6]:
def model(input_shape=(103, 50, 50, 1), classes=9):
    X_input = Input(input_shape)
    
    print(X_input.shape)
    X = feature_extraction(X_input)
    
    X = GlobalMaxPooling3D()(X)
    print(X.shape)
    X = Dense(classes, input_dim=256, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    model = Model(inputs = X_input, outputs = X, name="3D-SRNet")

    return model

In [7]:
model = model(input_shape = (103, 50, 50, 1), classes = 9)

(?, 103, 50, 50, 1)
(?, 48, 24, 24, 32)
(?, 48, 12, 12, 32)
(?, 48, 12, 12, 32)
--------
(?, 48, 12, 12, 16)
(?, 48, 12, 12, 16)
(?, 48, 12, 12, 16)
(?, 48, 12, 12, 16)
(?, 48, 12, 12, 32)
(?, 48, 12, 12, 32)
--------
(?, 48, 12, 12, 32)
(?, 48, 12, 12, 32)
--------
(?, 48, 12, 12, 32)
(?, 48, 12, 12, 32)
(?, 48, 12, 12, 32)
(?, 48, 12, 12, 32)
(?, 48, 12, 12, 64)
(?, 48, 12, 12, 64)
--------
(?, 48, 12, 12, 64)
(?, 48, 12, 12, 64)
(?, 64)


In [8]:
model.summary()

Model: "3D-SRNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 103, 50, 50,  0                                            
__________________________________________________________________________________________________
conv1 (Conv3D)                  (None, 48, 24, 24, 3 2336        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 48, 24, 24, 3 128         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 48, 24, 24, 3 0           bn_conv1[0][0]                   
___________________________________________________________________________________________

In [52]:
model.compile(optimizer= keras.optimizers.SGD(lr=0.01), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(Xtrain, Ytrain, epochs = 15, batch_size = 64)

Epoch 1/15
6300/6300 [==============================] - 32s 5ms/step - loss: 1.5710 - categorical_accuracy: 0.6040
Epoch 2/15
6300/6300 [==============================] - 24s 4ms/step - loss: 0.5520 - categorical_accuracy: 0.8113
Epoch 3/15
6300/6300 [==============================] - 24s 4ms/step - loss: 0.3575 - categorical_accuracy: 0.8800
Epoch 4/15
6300/6300 [==============================] - 24s 4ms/step - loss: 0.2504 - categorical_accuracy: 0.9208
Epoch 5/15
6300/6300 [==============================] - 24s 4ms/step - loss: 0.2024 - categorical_accuracy: 0.9379
Epoch 6/15
6300/6300 [==============================] - 24s 4ms/step - loss: 0.1534 - categorical_accuracy: 0.9590
Epoch 7/15
6300/6300 [==============================] - 24s 4ms/step - loss: 0.1289 - categorical_accuracy: 0.9662
Epoch 8/15
6300/6300 [==============================] - 24s 4ms/step - loss: 0.1100 - categorical_accuracy: 0.9735
Epoch 9/15
6300/6300 [==============================] - 25s 4ms/step - loss: 0.0

In [53]:
preds = model.evaluate(Xtest, Ytest)
# print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

26371/26371 [==============================] - 60s 2ms/step
Test Accuracy = 0.9618899548746729


In [54]:
y_pred = model.predict(Xtest, verbose=1)

26371/26371 [==============================] - 53s 2ms/step


In [55]:
import sklearn.metrics
confusion_matrix = sklearn.metrics.confusion_matrix(np.argmax(Ytest, axis=1), np.argmax(y_pred, axis=1))
confusion_matrix

array([[ 3935,    16,     1,   341,     0,    13,     0,    42,     0],
       [   23, 10868,     0,   192,     0,     0,     0,    14,    23],
       [    0,     0,   390,     0,     0,     0,     0,     0,     4],
       [   97,    34,     5,  1684,     0,     0,     0,    39,    12],
       [    0,     0,     0,     0,   639,     0,     0,     0,     6],
       [    0,     0,     0,     0,     0,  4277,     1,     0,    51],
       [    0,     0,     0,     0,     0,     0,   620,     0,    10],
       [    9,     1,     0,    58,     0,     0,     0,  2791,     2],
       [    0,     0,     3,     1,     3,     0,     4,     0,   162]],
      dtype=int64)

In [32]:
uSalinas = sio.loadmat('D:\\DEVANG_DATA\\EE_297_PROJECT\\TransferLearning\\Salinas\\Salinas_corrected.mat')
gt_uSalinas = sio.loadmat('D:\\DEVANG_DATA\\EE_297_PROJECT\\TransferLearning\\Salinas\\Salinas_gt.mat')
data_SA = uSalinas['salinas_corrected']
gt_SA = gt_uSalinas['salinas_gt']
print(data_SA.shape, gt_SA.shape)
data_SA = np.moveaxis(data_SA, 2, 0)
print(data_SA.shape, gt_SA.shape)

(512, 217, 204) (512, 217)
(204, 512, 217) (512, 217)


In [33]:
values,counts = np.unique(gt_SA, return_counts=True)
values,counts

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
       dtype=uint8),
 array([56975,  2009,  3726,  1976,  1394,  2678,  3959,  3579, 11271,
         6203,  3278,  1068,  1927,   916,  1070,  7268,  1807],
       dtype=int64))

In [7]:
len(data_SA[0][0])

217

In [42]:
small_segmented_1_SA = []
small_seg_gt_1_SA = []
Cube_size = 25
stride = 1
for i in range(Cube_size-1,len(data_SA[0]), stride):
    for j in range(Cube_size-1,len(data_SA[0][0]), stride):
        if gt_SA[(i-Cube_size+1) + int((Cube_size-1)/2+1)][(j-Cube_size+1) + int((Cube_size-1)/2+1)] != 0 and gt_SA[(i-Cube_size+1) + int((Cube_size-1)/2+1)][(j-Cube_size+1) + int((Cube_size-1)/2+1)] != 16:
            small_segmented_1_SA.append(np.asarray(data_SA[:, (i-Cube_size+1):i+1, (j-Cube_size+1):j+1]))
            small_seg_gt_1_SA.append(gt_SA[(i-Cube_size+1)+int(((Cube_size-1)/2)+1)][(j-Cube_size+1)+int(((Cube_size-1)/2)+1)])
small_segmented_1_SA = np.array(small_segmented_1_SA)
small_seg_gt_1_SA = np.array(small_seg_gt_1_SA)
small_segmented_1_SA.shape, small_seg_gt_1_SA.shape

((41851, 204, 50, 50), (41851,))

In [43]:
values,counts = np.unique(small_seg_gt_1_SA, return_counts=True)
values,counts

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
       dtype=uint8),
 array([1269, 3726, 1541, 1179, 2273, 3331, 2606, 9766, 5515, 2568,  813,
        1475,  725,  838, 4226], dtype=int64))

In [10]:
Lowest_number = np.min(counts)
Lowest_number

183

In [11]:
a = 23
indx1 = np.where(small_seg_gt_1_SA == 1)[0][:Lowest_number-a]
indxt1 =np.where(small_seg_gt_1_SA == 1)[0][Lowest_number-a:]
indx2 = np.where(small_seg_gt_1_SA == 2)[0][:Lowest_number-a]
indxt2 =  np.where(small_seg_gt_1_SA == 2)[0][Lowest_number-a:]
indx3 = np.where(small_seg_gt_1_SA == 3)[0][:Lowest_number-a]
indxt3 =  np.where(small_seg_gt_1_SA == 3)[0][Lowest_number-a:]
indx4 = np.where(small_seg_gt_1_SA == 4)[0][:Lowest_number-a]
indxt4 =  np.where(small_seg_gt_1_SA == 4)[0][Lowest_number-a:]
indx5 = np.where(small_seg_gt_1_SA == 5)[0][:Lowest_number-a]
indxt5 =  np.where(small_seg_gt_1_SA == 5)[0][Lowest_number-a:]
indx6 = np.where(small_seg_gt_1_SA == 6)[0][:Lowest_number-a]
indxt6 =  np.where(small_seg_gt_1_SA == 6)[0][Lowest_number-a:]
indx7 = np.where(small_seg_gt_1_SA == 7)[0][:Lowest_number-a]
indxt7 =  np.where(small_seg_gt_1_SA == 7)[0][Lowest_number-a:]
indx8 = np.where(small_seg_gt_1_SA == 8)[0][:Lowest_number-a]
indxt8 =  np.where(small_seg_gt_1_SA == 8)[0][Lowest_number-a:]
indx9 = np.where(small_seg_gt_1_SA == 9)[0][:Lowest_number-a]
indxt9 =  np.where(small_seg_gt_1_SA == 9)[0][Lowest_number-a:]
indx10 = np.where(small_seg_gt_1_SA == 10)[0][:Lowest_number-a]
indxt10 =np.where(small_seg_gt_1_SA == 10)[0][Lowest_number-a:]
indx11 = np.where(small_seg_gt_1_SA == 11)[0][:Lowest_number-a]
indxt11 =  np.where(small_seg_gt_1_SA == 11)[0][Lowest_number-a:]
indx12 = np.where(small_seg_gt_1_SA == 12)[0][:Lowest_number-a]
indxt12 =  np.where(small_seg_gt_1_SA == 12)[0][Lowest_number-a:]
indx13 = np.where(small_seg_gt_1_SA == 13)[0][:Lowest_number-a]
indxt13 =  np.where(small_seg_gt_1_SA == 13)[0][Lowest_number-a:]
indx14 = np.where(small_seg_gt_1_SA == 14)[0][:Lowest_number-a]
indxt14 =  np.where(small_seg_gt_1_SA == 14)[0][Lowest_number-a:]
indx15 = np.where(small_seg_gt_1_SA == 15)[0][:Lowest_number-a]
indxt15 =  np.where(small_seg_gt_1_SA == 15)[0][Lowest_number-a:]
indx1.shape, indx2.shape, indx3.shape, indx4.shape, indx5.shape, indx6.shape, indx7.shape, indx8.shape, indx9.shape

((160,), (160,), (160,), (160,), (160,), (160,), (160,), (160,), (160,))

In [12]:
Xtrain = []
Xtest = []
Ytrain = []
Ytest = []
list_of_indx = [indx1,indx2,indx3,indx4,indx5,indx6,indx7,indx8,indx9,indx10,indx11,indx12,indx13,indx14,indx15]
list_of_indxt = [indxt1,indxt2,indxt3,indxt4,indxt5,indxt6,indxt7,indxt8,indxt9,indxt10,indxt11,indxt12,indxt13,indxt14,indxt15]
for i in range(len(list_of_indx)):
    for j in list_of_indx[i]:
        Xtrain.append(small_segmented_1_SA[j])
        Ytrain.append(small_seg_gt_1_SA[j])
    for k in list_of_indxt[i]:
        Xtest.append(small_segmented_1_SA[k])
        Ytest.append(small_seg_gt_1_SA[k])

Xtrain = np.array(Xtrain)
Xtest = np.array(Xtest)
Ytrain = np.array(Ytrain)
Ytest = np.array(Ytest)
Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape

((2400, 204, 50, 50), (8112, 204, 50, 50), (2400,), (8112,))

In [13]:
s = np.arange(Xtrain.shape[0])
np.random.shuffle(s)
Xtrain = Xtrain[s]
Ytrain = Ytrain[s]

In [14]:
s = np.arange(Xtest.shape[0])
np.random.shuffle(s)
Xtest = Xtest[s]
Ytest = Ytest[s]
Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape

((2400, 204, 50, 50), (8112, 204, 50, 50), (2400,), (8112,))

In [15]:
Xtrain = np.expand_dims(Xtrain,axis=4) 
Xtest = np.expand_dims(Xtest,axis=4) 
Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape

((2400, 204, 50, 50, 1), (8112, 204, 50, 50, 1), (2400,), (8112,))

In [16]:
np.unique(Ytrain, return_counts=True), np.unique(Ytest, return_counts=True)

((array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
        dtype=uint8),
  array([160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160,
         160, 160], dtype=int64)),
 (array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
        dtype=uint8),
  array([ 165,  772,  219,  127,  406,  678,  489, 2293, 1232,  485,   46,
          212,   23,   52,  913], dtype=int64)))

In [17]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
Ytrain = Ytrain.reshape(len(Ytrain), 1)
Ytest = Ytest.reshape(len(Ytest),1)
Ytrain = onehot_encoder.fit_transform(Ytrain)
Ytest = onehot_encoder.fit_transform(Ytest)
Ytrain[0], Ytest[0]

C:\Users\devang1997\software\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\devang1997\software\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categori

(array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]))

In [22]:
def model(input_shape=(204, 50, 50, 1), classes=9):
    X_input = Input(input_shape)
    
    print(X_input.shape)
    X = feature_extraction(X_input)
    
    X = GlobalMaxPooling3D()(X)
    print(X.shape)
    X = Dense(classes, input_dim=256, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    model = Model(inputs = X_input, outputs = X, name="3D-SRNet")

    return model

In [23]:
model = model(input_shape = (204, 50, 50, 1), classes = 15)
model.summary()

(?, 204, 50, 50, 1)
(?, 99, 24, 24, 32)
(?, 99, 12, 12, 32)
(?, 99, 12, 12, 32)
--------
(?, 99, 12, 12, 16)
(?, 99, 12, 12, 16)
(?, 99, 12, 12, 16)
(?, 99, 12, 12, 16)
(?, 99, 12, 12, 32)
(?, 99, 12, 12, 32)
--------
(?, 99, 12, 12, 32)
(?, 99, 12, 12, 32)
--------
(?, 99, 12, 12, 32)
(?, 99, 12, 12, 32)
(?, 99, 12, 12, 32)
(?, 99, 12, 12, 32)
(?, 99, 12, 12, 64)
(?, 99, 12, 12, 64)
--------
(?, 99, 12, 12, 64)
(?, 99, 12, 12, 64)
(?, 64)
Model: "3D-SRNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 204, 50, 50,  0                                            
__________________________________________________________________________________________________
conv1 (Conv3D)                  (None, 99, 24, 24, 3 2336        input_2[0][0]                    
___________________________________________

In [24]:
model.compile(optimizer= keras.optimizers.SGD(lr=0.001), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(Xtrain, Ytrain, epochs = 30, batch_size = 64)

Epoch 1/10
2400/2400 [==============================] - 20s 9ms/step - loss: 4.9436 - categorical_accuracy: 0.2450
Epoch 2/10
2400/2400 [==============================] - 17s 7ms/step - loss: 0.6669 - categorical_accuracy: 0.8292
Epoch 3/10
2400/2400 [==============================] - 18s 7ms/step - loss: 0.1708 - categorical_accuracy: 0.9712
Epoch 4/10
2400/2400 [==============================] - 17s 7ms/step - loss: 0.0825 - categorical_accuracy: 0.9950
Epoch 5/10
2400/2400 [==============================] - 18s 7ms/step - loss: 0.0553 - categorical_accuracy: 0.9963
Epoch 6/10
2400/2400 [==============================] - 17s 7ms/step - loss: 0.0433 - categorical_accuracy: 0.9971
Epoch 7/10
2400/2400 [==============================] - 18s 7ms/step - loss: 0.0323 - categorical_accuracy: 0.9992
Epoch 8/10
2400/2400 [==============================] - 18s 7ms/step - loss: 0.0246 - categorical_accuracy: 0.9996
Epoch 9/10
2400/2400 [==============================] - 18s 7ms/step - loss: 0.0

In [25]:
preds = model.evaluate(Xtest, Ytest)
# print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

8112/8112 [==============================] - 35s 4ms/step
Test Accuracy = 0.6744329388560157


In [26]:
y_pred = model.predict(Xtest, verbose=1)

8112/8112 [==============================] - 33s 4ms/step


In [27]:
import sklearn.metrics
confusion_matrix = sklearn.metrics.confusion_matrix(np.argmax(Ytest, axis=1), np.argmax(y_pred, axis=1))
confusion_matrix

array([[ 165,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [ 279,   52,    0,    0,    0,    4,  246,    0,    0,    0,  117,
          74,    0,    0,    0],
       [   4,   41,  173,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    1,    0,    0],
       [   0,    0,    9,  117,    0,    0,    0,    1,    0,    0,    0,
           0,    0,    0,    0],
       [   0,    0,   15,    5,  386,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   0,    0,    0,    0,   24,  644,    0,    0,    0,   10,    0,
           0,    0,    0,    0],
       [   0,    0,    6,    0,    0,    4,  434,    0,    0,   34,    7,
           0,    4,    0,    0],
       [  23,  301,  214,    2,    0,    2,    4, 1638,    0,    0,    0,
           0,    1,    2,  106],
       [   0,    0,  331,   14,    0,    0,    0,   33,  802,    3,    0,
           9,    2,    4,   34],
       [   0,    0,   84,    0,    0,

In [22]:
uIndian_pines = sio.loadmat('D:\\DEVANG_DATA\\EE_297_PROJECT\\TransferLearning\\Indian Pines\\Indian_pines_corrected.mat')
gt_uIndian_pines = sio.loadmat('D:\\DEVANG_DATA\\EE_297_PROJECT\\TransferLearning\\Indian Pines\\Indian_pines_gt.mat')
data_IN = uIndian_pines['indian_pines_corrected']
gt_IN = gt_uIndian_pines['indian_pines_gt']
print(data_IN.shape, gt_IN.shape)
data_IN = np.moveaxis(data_IN, 2, 0)
print(data_IN.shape, gt_IN.shape)

(145, 145, 200) (145, 145)
(200, 145, 145) (145, 145)


In [23]:
values,counts = np.unique(gt_IN, return_counts=True)
values,counts

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
       dtype=uint8),
 array([10776,    46,  1428,   830,   237,   483,   730,    28,   478,
           20,   972,  2455,   593,   205,  1265,   386,    93],
       dtype=int64))

In [24]:
len(data_IN[0][0])

145

In [29]:
small_segmented_1_IN = []
small_seg_gt_1_IN = []
Cube_size = 25
stride = 1
for i in range(Cube_size-1,len(data_IN[0]), stride):
    for j in range(Cube_size-1,len(data_IN[0][0]), stride):
        if gt_IN[(i-Cube_size+1) + int((Cube_size-1)/2+1)][(j-Cube_size+1) + int((Cube_size-1)/2+1)] != 0:# and gt_IN[(i-Cube_size+1) + int((Cube_size-1)/2+1)][(j-Cube_size+1) + int((Cube_size-1)/2+1)] != 1 and gt_IN[(i-Cube_size+1) + int((Cube_size-1)/2+1)][(j-Cube_size+1) + int((Cube_size-1)/2+1)] != 7 and gt_IN[(i-Cube_size+1) + int((Cube_size-1)/2+1)][(j-Cube_size+1) + int((Cube_size-1)/2+1)] != 9:
            small_segmented_1_IN.append(np.asarray(data_IN[:, (i-Cube_size+1):i+1, (j-Cube_size+1):j+1]))
            small_seg_gt_1_IN.append(gt_IN[(i-Cube_size+1)+int(((Cube_size-1)/2)+1)][(j-Cube_size+1)+int(((Cube_size-1)/2)+1)])
small_segmented_1_IN = np.array(small_segmented_1_IN)
small_seg_gt_1_IN = np.array(small_seg_gt_1_IN)
small_segmented_1_IN.shape, small_seg_gt_1_IN.shape

((8181, 200, 25, 25), (8181,))

In [30]:
values,counts = np.unique(small_seg_gt_1_IN, return_counts=True)
values,counts

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
       dtype=uint8),
 array([  46, 1348,  442,  105,  289,  730,   28,  331,   20,  825, 2182,
         413,  205, 1025,   99,   93], dtype=int64))

In [31]:
Lowest_number = np.min(counts)
Lowest_number

20